In [ ]:
from pyrosm import OSM
from pyrosm import get_data
import openeo
import matplotlib.pyplot as plt

import os

In [ ]:
city = "Bonn"

USE_LOCAL = False

In [ ]:

if not os.path.exists(os.path.join('data', city)):
    USE_LOCAL = False
if not USE_LOCAL:
    os.makedirs(os.path.join('data', city), exist_ok=True)

In [ ]:
# Download data for given city, or load if already downloaded
fp = get_data(city, directory=os.path.join('data', city))
osm = OSM(fp)
print("Loaded OSM data for " + city)

# Filter buildings
buildings = osm.get_buildings()
print("Filtered buildings in " + city)

# Get boundaries of the city
min_x, min_y, max_x, max_y = buildings.total_bounds
print("Coordinates of " + city + ": " + str(min_x),
      str(min_y), str(max_x), str(max_y))

In [ ]:
connection = openeo.connect("https://openeo.cloud")
# authenticate
connection.authenticate_oidc()
collections = connection.list_collections()
collections

In [ ]:
connection.describe_collection("SENTINEL2_L2A")

In [ ]:
import openeo.util
from openeo.processes import ProcessBuilder
def scale_function(x: ProcessBuilder):
    return x.linear_scale_range(0, 5000, 0, 255)

if not USE_LOCAL:  
    

    # define time
    temporal_extent = ["2023-05-01", "2023-09-30"]

    # define coordinates
    bbox = {"west": min_x, "south": min_y, "east": max_x, "north": max_y}

    # define the collection ID
    collection_id = "SENTINEL2_L2A"

    # define bands
    # ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'SCL', 'SNW', 'CLD', 'CLP', 'CLM', 'sunAzimuthAngles', 'sunZenithAngles', 'viewAzimuthMean', 'viewZenithMean', 'dataMask']
    bands = ["B04", "B03", "B02", ]

    # load the data collection
    datacube = connection.load_collection(
        collection_id=collection_id,
        spatial_extent=bbox,
        temporal_extent=temporal_extent,
        bands=bands,
        max_cloud_cover=10
    )

    # merge temporal dimension
    # datacube = datacube.reduce_temporal("median")

    # blue = datacube.band("B02") * 0.0001
    # red = datacube.band("B04") * 0.0001
    # green = datacube.band("B03") * 0.0001
    
    # blue = blue.reduce_temporal("median")
    # red = red.reduce_temporal("median")
    # green = green.reduce_temporal("median")

    # datacube.band("B02", blue)
    # datacube.band("B04", red)
    # datacube.band("B03", green)

    # display(blue)

    # display the data
    datacube_scale = datacube.apply(scale_function)
    datacube_scale_reduced = datacube_scale.reduce_temporal("median")

    # create cloud mask


    display(datacube_scale_reduced)
    # save the data as TIFF files
    datacube_scale_reduced.download(os.path.join(
        "data", city, "satellite_data.tiff"), format="GTiff")

In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np
import plotly.express as px

path = "./data/Bonn/satellite_data.tiff"

with rasterio.open(path) as src:
    data = src.read()
    
    data = np.moveaxis(data, [0], [2])
    profile = src.profile
    print(profile)
    # plot matplotlib

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(data)
    plt.show()

# plot plotly
# fig = px.imshow(np.moveaxis(np.array(data), [0], [2]))
# fig


In [ ]:
from rasterio.features import geometry_mask
mask = geometry_mask(buildings.geometry, data.shape[:-1], profile["transform"], all_touched=False)

In [ ]:
px.imshow(mask)

In [ ]:
buildings.plot()